In [3]:
import gensim
import MeCab
import numpy as np
from scipy import spatial

# word2vec_model = gensim.models.KeyedVectors.load_word2vec_format('./dataset/model.vec', binary=False)
word2vec_model = gensim.models.KeyedVectors.load_word2vec_format('./dataset/entity_vector.model.bin', binary=True)
# word2vec_model = gensim.models.Word2Vec.load('./dataset/ja.bin')
# word2vec_model = gensim.models.Word2Vec.load('./dataset/entity_vector.model.bin')

# mecab = MeCab.Tagger("-d /usr/local/lib/mecab/dic/mecab-ipadic-neologd -Owakati")
# mecab = MeCab.Tagger("-d /var/lib/mecab/dic/ipadic-utf8 -Owakati")
mecab = MeCab.Tagger("-d /var/lib/mecab/dic/ipadic-utf8")
# mecab = MeCab.Tagger("")

def avg_feature_vector(noun_bag, model, num_features): 
    feature_vec = np.zeros((num_features,), dtype="float32") 
    for noun in noun_bag:
        try:
            feature_vec = np.add(feature_vec, model[noun])
        except KeyError:
            print("There is no key: " + noun)
    if len(noun_bag) > 0:
        feature_vec = np.divide(feature_vec, len(noun_bag))
    return feature_vec

# ２つの文章の類似度を算出
def sentence_similarity(sentence1, sentence2):
    
    parse_results1 = mecab.parse(sentence1)
    parse_results2 = mecab.parse(sentence2)
    
    noun_bag1 = []
    for res in parse_results1.split('\n'):
        cols = res.split('\t')
        if(1 < len(cols)):
            parts = cols[1].split(',')
            if(parts[0].startswith('名詞')):
                noun_bag1.append(cols[0])

    noun_bag2 = []
    for res in parse_results2.split('\n'):
        cols = res.split('\t')
        if(1 < len(cols)):
            parts = cols[1].split(',')
            if(parts[0].startswith('名詞')):
                noun_bag2.append(cols[0])

    
    # num_features=300
    num_features = 200
    sentence_1_avg_vector = avg_feature_vector(noun_bag1, word2vec_model, num_features)
    sentence_2_avg_vector = avg_feature_vector(noun_bag2, word2vec_model, num_features)
    return 1 - spatial.distance.cosine(sentence_1_avg_vector, sentence_2_avg_vector)

result = sentence_similarity(
    "彼は昨日、激辛ラーメンを食べてお腹を壊した",
    "昨日、僕も激辛の中華料理を食べてお腹を壊した"
)
print(result)


/home/djinn/.local/lib/python3.5/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


0.8419689536094666


In [6]:
import csv

with open("./dataset/H30_kagaku.csv") as f:
    reader = csv.reader(f)
    data = [row for row in reader]

student_id = 4
sentences = data[student_id][8].split("。")

sim_vec = []
for i in range(0, len(sentences)-1):
    sentence1 = sentences[i]
    sentence2 = sentences[i + 1]
    
    sim_val = sentence_similarity(sentence1, sentence2)
    sim_vec.append(sim_val) 
# print(sim_vec)
    
print(sentences[0])
for i in range(0, len(sentences)-1):
    print(sim_vec[i])
    print(sentences[i + 1])

【研究の動機】私は今学校でクイズ研究会に所属しています
0.443587064743042
この部活では、主に早押しクイズをしており、知識だけでなく早さ（速さ）も求められます
0.3979988098144531
私は「自分がわかった瞬間」にボタンを押す、という反応が鈍いため、この早押しクイズが苦手です
0.8276987671852112
そこで、自分と押し勝てる（クイズで、他の人よりボタンを早く押し解答権を得ることを「押し勝つ」といいます）人とではどのような違いがあるのか知りたいと思いました
0.7742905020713806
【研究の目的】人が刺激を受けてから反応するまでの時間を、条件を変えながら測定することで、何の違いによって人の反応速度は変化するのか、ということを明らかにしたい
0.876459002494812
また、実験結果から、人の反応速度をあげるにはどうすればよいかの方法も考察したい
0.6754539012908936
「実験」では、身体のコンディションが違う人たちのグループ（A）と、個人の生まれ持ったものや能力が違う人たちのグループ（B）の大きく２グループに分け、全ての実験を２グループとも行うものとする
0.5330678224563599
（A）A1.空腹かどうか｛a　食後すぐの人（１０人）
0.7839764952659607
b　食後５時間が経ち、空腹の人（１０人）
0.8921685814857483
｝A2.眠いかどうか｛a　７時間睡眠をとった人（１０人）
0.8017783761024475
b　徹夜明けの人（１０人）
0.7758699655532837
｝A3.疲れているかどうか｛a　７時間睡眠をとった人【疲労感なし】（１０人）
0.8188786506652832
b　１時間計算問題を解き続けた人【精神的疲労】（１０人）
0.9465337991714478
c　１時間運動し続けた人【肉体的疲労】（１０人）
0.5874164700508118
｝（B）B1.男女の違い｛a　男性（１０人）
0.7504124045372009
b　女性（１０人）
0.7552652359008789
｝B2.運動能力の違い｛a　スポーツ競技経験が１年以上ある人（１０人）
0.7297317385673523
b　特に何もしていない人（１０人